## Batch attributes from existing tables using Signals

This notebook creates a new view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    wh[(Warehouse)]
    cron[/Materialization CRON job/]
    signals(Signals)

    wh --> cron
    cron --> signals
```

---

# Installation and setup

In [ ]:
%pip install snowplow-signals

In [ ]:
from snowplow_signals import Signals
import os

try:
    from google.colab import userdata
    sp_signals = Signals(
            api_url=userdata.get('SP_API_URL'),
            api_key=userdata.get('SP_API_KEY'),
            api_key_id=userdata.get('SP_API_KEY_ID'),
            org_id=userdata.get('SP_ORG_ID'),
        )
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
    sp_signals = Signals(
        api_url=os.environ['SP_API_URL'],
        api_key=os.environ['SP_API_KEY'],
        api_key_id=os.environ['SP_API_KEY_ID'],
        org_id=os.environ['SP_ORG_ID'],
)

### Define a new data source

Creates a data source with the source Snowflake table configuration.

In [ ]:
from snowplow_signals import BatchSource

data_source = BatchSource(
    name="ecommerce_transaction_interactions_source",
    database="SNOWPLOW_DEV1",
    schema="SIGNALS",
    table="SNOWPLOW_ECOMMERCE_TRANSACTION_INTERACTIONS_FEATURES",
    timestamp_field="UPDATED_AT",
)

### Create a view with the table fields

Views define the attributes inside the data sources and the entity types they relate to.

In [ ]:
from snowplow_signals import View, domain_userid, Field

view = View(
    name="ecommerce_transaction_interactions_attributes",
    version=1,
    entity=domain_userid,
    fields=[
        Field(
            name="TOTAL_TRANSACTIONS",
            type="int32",
        ),
        Field(
            name="TOTAL_REVENUE",
            type="int32",
        ),
        Field(
            name="AVG_TRANSACTION_REVENUE",
            type="int32",
        ),
    ],
    batch_source=data_source,
    owner="user@company.com",
)

### Applying the data source and view to Signals

The following block pushes the data source and view definition to the Signals API and makes it available for a background CRON job that incrementally materializes the data from the warehouse table to the online attribute store.

In [ ]:
from snowplow_signals import Signals

applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")

### Retrieving data

One can fetch the latest attribute values for a particular user from the view as follow.

In [ ]:
response = view.get_attributes(
    signals=sp_signals,
    identifier="9999999999999999999999999",
)

response